# importing the libraries

In [91]:
import numpy as np
import pandas as pd

# loading the dataset 

In [92]:
movie=pd.read_csv('https://raw.githubusercontent.com/rajeevratan84/datascienceforbusiness/refs/heads/master/MoviesOnStreamingPlatforms_updated.csv')




# understanding the dataset

In [93]:
movie.columns

Index(['Unnamed: 0', 'ID', 'Title', 'Year', 'Age', 'IMDb', 'Rotten Tomatoes',
       'Netflix', 'Hulu', 'Prime Video', 'Disney+', 'Type', 'Directors',
       'Genres', 'Country', 'Language', 'Runtime'],
      dtype='object')

In [94]:
movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16744 entries, 0 to 16743
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       16744 non-null  int64  
 1   ID               16744 non-null  int64  
 2   Title            16744 non-null  object 
 3   Year             16744 non-null  int64  
 4   Age              7354 non-null   object 
 5   IMDb             16173 non-null  float64
 6   Rotten Tomatoes  5158 non-null   object 
 7   Netflix          16744 non-null  int64  
 8   Hulu             16744 non-null  int64  
 9   Prime Video      16744 non-null  int64  
 10  Disney+          16744 non-null  int64  
 11  Type             16744 non-null  int64  
 12  Directors        16018 non-null  object 
 13  Genres           16469 non-null  object 
 14  Country          16309 non-null  object 
 15  Language         16130 non-null  object 
 16  Runtime          16152 non-null  float64
dtypes: float64(2

# Data cleaning

In [95]:
movie=movie.drop(columns=['Unnamed: 0']) # removing the unnecessary columns

In [96]:
movie['Age']=movie['Age'].fillna(movie['Age'].mode()[0])
movie['IMDb']=movie['IMDb'].fillna(movie['IMDb'].mean()).round(1)

movie['Rotten Tomatoes']=movie['Rotten Tomatoes'].str.strip('%')
movie['Rotten Tomatoes']=movie['Rotten Tomatoes'].astype(float)
movie['Rotten Tomatoes']=(movie['Rotten Tomatoes'].fillna(movie['Rotten Tomatoes'].mean())/10).round(1)

In [97]:
movie['Directors']=movie['Directors'].fillna('unknown')
movie['Genres']=movie['Genres'].fillna('unknown')
movie['Country']=movie['Country'].fillna('unknown')
movie['Language']=movie['Language'].fillna('unknown')
movie['Runtime']=movie['Runtime'].fillna(movie['Runtime'].mean())

In [98]:
movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16744 entries, 0 to 16743
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               16744 non-null  int64  
 1   Title            16744 non-null  object 
 2   Year             16744 non-null  int64  
 3   Age              16744 non-null  object 
 4   IMDb             16744 non-null  float64
 5   Rotten Tomatoes  16744 non-null  float64
 6   Netflix          16744 non-null  int64  
 7   Hulu             16744 non-null  int64  
 8   Prime Video      16744 non-null  int64  
 9   Disney+          16744 non-null  int64  
 10  Type             16744 non-null  int64  
 11  Directors        16744 non-null  object 
 12  Genres           16744 non-null  object 
 13  Country          16744 non-null  object 
 14  Language         16744 non-null  object 
 15  Runtime          16744 non-null  float64
dtypes: float64(3), int64(7), object(6)
memory usage: 2.0+ MB


In [99]:
movie.head(2)

,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,1,Inception,2010,13+,8.8,8.7,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,2,The Matrix,1999,18+,8.7,8.7,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0


In [100]:
x=movie['Genres'].str.split(',')
Genres=x.explode()

Genres_df=pd.merge(movie[['Title','IMDb']],Genres,left_index=True,right_index=True)
Genres_df.head()

,Title,IMDb,Genres
0,Inception,8.8,Action
0,Inception,8.8,Adventure
0,Inception,8.8,Sci-Fi
0,Inception,8.8,Thriller
1,The Matrix,8.7,Action


In [101]:
# Splitting 'Language' column where multiple languages are separated by commas
language_split = movie['Language'].str.split(',')
Languages = language_split.explode()

# Creating a DataFrame with Title, IMDb, and Language
Languages_df = pd.merge(movie[['Title', 'IMDb']], Languages, left_index=True, right_index=True)
Languages_df.head()


,Title,IMDb,Language
0,Inception,8.8,English
0,Inception,8.8,Japanese
0,Inception,8.8,French
1,The Matrix,8.7,English
2,Avengers: Infinity War,8.5,English


In [102]:
# Melting the dataset to transform platform columns into rows
Platforms_df = movie.melt(id_vars=['Title', 'IMDb'], 
                          value_vars=['Netflix', 'Hulu', 'Prime Video', 'Disney+'], 
                          var_name='Platform', 
                          value_name='Available')

# Filtering out movies that are NOT available (Available == 0)
Platforms_df = Platforms_df[Platforms_df['Available'] == 1].drop(columns=['Available'])

Platforms_df.head()


,Title,IMDb,Platform
0,Inception,8.8,Netflix
1,The Matrix,8.7,Netflix
2,Avengers: Infinity War,8.5,Netflix
3,Back to the Future,8.5,Netflix
4,"The Good, the Bad and the Ugly",8.8,Netflix


In [103]:
# Splitting 'Directors' column
directors_split = movie['Directors'].str.split(',')
Directors = directors_split.explode()

# Creating Directors_df
Directors_df = pd.merge(movie[['Title', 'IMDb']], Directors, left_index=True, right_index=True)
Directors_df.head()

,Title,IMDb,Directors
0,Inception,8.8,Christopher Nolan
1,The Matrix,8.7,Lana Wachowski
1,The Matrix,8.7,Lilly Wachowski
2,Avengers: Infinity War,8.5,Anthony Russo
2,Avengers: Infinity War,8.5,Joe Russo


In [104]:
# Defining runtime categories
bins = [0, 60, 120, 180, 300]  # (in minutes)
labels = ['Short (<60 min)', 'Medium (60-120 min)', 'Long (120-180 min)', 'Very Long (>180 min)']
movie['Runtime Category'] = pd.cut(movie['Runtime'], bins=bins, labels=labels, right=False)


In [106]:
movie

,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime,Runtime Category
0,1,Inception,2010,13+,8.8,8.7,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.000000,Long (120-180 min)
1,2,The Matrix,1999,18+,8.7,8.7,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.000000,Long (120-180 min)
2,3,Avengers: Infinity War,2018,13+,8.5,8.4,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.000000,Long (120-180 min)
3,4,Back to the Future,1985,7+,8.5,9.6,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.000000,Medium (60-120 min)
4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,9.7,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.000000,Long (120-180 min)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16739,16740,The Ghosts of Buxley Hall,1980,18+,6.2,6.5,0,0,0,1,0,Bruce Bilson,"Comedy,Family,Fantasy,Horror",United States,English,120.000000,Long (120-180 min)
16740,16741,The Poof Point,2001,7+,4.7,6.5,0,0,0,1,0,Neal Israel,"Comedy,Family,Sci-Fi",United States,English,90.000000,Medium (60-120 min)
16741,16742,Sharks of Lost Island,2013,18+,5.7,6.5,0,0,0,1,0,Neil Gelinas,Documentary,United States,English,93.413447,Medium (60-120 min)
16742,16743,Man Among Cheetahs,2017,18+,6.6,6.5,0,0,0,1,0,Richard Slater-Jones,Documentary,United States,English,93.413447,Medium (60-120 min)


# DATA ANALYSIS

# How many unique movies are in the dataset?
# What is the average IMDb rating of all movies?
# How many movies were released each year?
# What is the most common genre in the dataset?
# What is the most common language for movies?

# How many movies are available on each streaming platform?
# Which platform has the highest average IMDb rating?
# Are there any movies available on all four platforms (Netflix, Hulu, Prime Video, Disney+)?
# Which platform has the most movies with an IMDb rating above 8.0?
# Which platform has the most movies released after 2015?

# What are the top 5 genres with the most movies?
# Which genre has the highest average IMDb rating?
# What is the most popular genre on Netflix?
# How many movies in the Comedy genre have an IMDb rating above 7.5?
# What is the average IMDb rating for Horror movies?

# Who are the top 5 directors with the most movies in the dataset?
# Which director has the highest average IMDb rating?
# Which director has made the most Action movies?
# What is the average IMDb rating of movies directed by Christopher Nolan?
# How many movies by Steven Spielberg are available on Prime Video?

# What are the top 5 most common languages in the dataset?
# What is the average IMDb rating of French movies?
# How many movies are from the United States?
# Which country has produced the most Drama movies?
# What is the most common language for movies on Hulu?
# IMDb and Rotten Tomatoes Score Analysis
# What is the highest IMDb rating in the dataset, and which movie has it?
# What is the lowest Rotten Tomatoes score, and which movie has it?
# What is the correlation between IMDb ratings and Rotten Tomatoes scores?
# How many movies have both IMDb rating above 8.0 and Rotten Tomatoes score above 90%?
# Which platform has the highest average Rotten Tomatoes score?

# What is the average runtime of all movies?
# Which movie has the longest runtime?
# Which genre tends to have the longest runtime on average?
# How many movies have a runtime of less than 60 minutes?
# What is the average IMDb rating for movies with a runtime above 150 minutes?